<a href="https://colab.research.google.com/github/LauraJouLee/Fall2021-Course-Project/blob/main/DH_Final_Hot100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install beautifulsoup4 pandas lxml future

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime, date, timedelta


URL = 'https://www.billboard.com/charts/hot-100/'

date_datum = date(2021,11,6)

link_list = []
date_list = []

for year in range(2019,2022): 
    if year == 2021:
        week_count_i = (abs(date_datum - date(year, 1, 1)).days)//7
        for i in range(0, week_count_i):
            link = URL + (str(date_datum - timedelta(7)*i))
            link_list.append(link)
            date_list.append(str(date_datum - timedelta(7)*i))
    else :
        week_count_j = (abs(date(year,12,31)-date(year, 1, 1)).days)//7
        for j in range(0, week_count_j):
            link = URL + (str(date(year,12,31) - timedelta(7)*j))
            link_list.append(link)
            date_list.append(str(date(year,12,31) - timedelta(7)*j))


pd.set_option('display.max_colwidth', None) #show full link
link = pd.DataFrame({'link': link_list, 'date': date_list})

link_chart = pd.concat([link], ignore_index=True).sort_values('date').reset_index(drop=True)


In [ ]:
import re

date_list = []
rank_list = []
song_list = []
artist_list = []


for url in link_list:
    req = requests.get(url)
    soup = bs(req.text, 'html.parser')

    for date in soup.find_all('link', href=re.compile('^/charts/hot-100/')):
        date_text = str(date.get('href')).replace('/charts/hot-100/','')  
        date_list.append(date_text)
        
    for rank in soup.find_all('span','chart-element__rank__number'):
        rank_list.append(str(rank.get_text()))
        
    for song in soup.find_all('span','chart-element__information__song text--truncate color--primary'):
        song_list.append(str(song.get_text()))
        
    for artist in soup.find_all('span','chart-element__information__artist text--truncate color--secondary'):
        artist_list.append(str(artist.get_text()))


date_full_list =  [d for d in date_list for i in range(100)]
      
chart = pd.DataFrame(
    {'date': date_full_list, 
     'rank': rank_list, 
     'song': song_list, 
     'artist': artist_list})


DH_Final_Hot100 = pd.merge(link_chart,chart, on = 'date') #value in both 'date' must be the same == str

In [ ]:
DH_Final_Hot100.to_csv('DH_Final_Hot100.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
